<a href="https://colab.research.google.com/github/yastika/myColabProjects/blob/main/Derivative_Pricing_GWP3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import scipy.stats as ss
import pandas as pd

## **Step 1**

In [ ]:
S0 = 80
r = 0.055
sigma = 0.35
T = 1/4 # 3 months

### Stochastic Volatility Modeler

In [ ]:
v0 = 0.032
kappa = 1.85
theta = 0.045
M=int(T * 255)
Ite = 1000000
dt = T/M

In [ ]:
def SDE_vol(v0, kappa, theta, sigma, T, M, Ite, rand, row, cho_matrix):
    dt = T / M  # T = maturity, M = number of time steps
    v = np.zeros((M + 1, Ite), dtype=float)
    v[0] = v0
    sdt = np.sqrt(dt)  # Sqrt of dt
    for t in range(1, M + 1):
        ran = np.dot(cho_matrix, rand[:, t])
        v[t] = np.maximum(
            0,
            v[t - 1]
            + kappa * (theta - v[t - 1]) * dt
            + np.sqrt(v[t - 1]) * sigma * ran[row] * sdt,
        )
    return v

In [ ]:
def Heston_paths(S0, r, v, row, cho_matrix):
    S = np.zeros((M + 1, Ite), dtype=float)
    S[0] = S0
    sdt = np.sqrt(dt)
    for t in range(1, M + 1, 1):
        ran = np.dot(cho_matrix, rand[:, t])
        S[t] = S[t - 1] * np.exp((r - 0.5 * v[t]) * dt + np.sqrt(v[t]) * ran[row] * sdt)

    return S

In [ ]:
def random_number_gen(M, Ite):
    rand = np.random.standard_normal((2, M + 1, Ite))
    return rand

In [ ]:
def cho_matrix_gen(rho):
  # Covariance Matrix
  covariance_matrix = np.zeros((2, 2), dtype=float)
  covariance_matrix[0] = [1.0, rho]
  covariance_matrix[1] = [rho, 1.0]
  cho_matrix = np.linalg.cholesky(covariance_matrix)

  return cho_matrix

In [ ]:
def heston_call_put_mc(S, K, r, T, t, opttype):
    payoff = np.zeros((M + 1, Ite), dtype=float)

    if opttype == 'C':
      payoff = np.maximum(0, S[-1, :] - K)

    else:
      payoff = np.maximum(0, K - S[-1, :])

    average = np.mean(payoff)

    return np.exp(-r * (T - t)) * average

#def heston_put_mc(S, K, r, T, t):
#    payoff = np.maximum(0, K - S[-1, :])

#    average = np.mean(payoff)

#    return np.exp(-r * (T - t)) * average*/

In [ ]:
# Generating random numbers from standard normal
rand = random_number_gen(M, Ite)
rho_1=-0.3
rho_2=-0.7
# Covariance Matrix
cho_matrix_1 = cho_matrix_gen(rho_1)
cho_matrix_2 = cho_matrix_gen(rho_2)

In [ ]:
# Volatility process paths and Underlying price process paths when correlation is -0.3
V_1 = SDE_vol(v0, kappa, theta, sigma, T, M, Ite, rand, 1, cho_matrix_1)
S_1 = Heston_paths(S0, r, V_1, 0, cho_matrix_1)

# Volatility process paths and Underlying price process paths when correlation is -0.7
V_2 = SDE_vol(v0, kappa, theta, sigma, T, M, Ite, rand, 1, cho_matrix_2)
S_2 = Heston_paths(S0, r, V_2, 0, cho_matrix_2)

In [ ]:
#print(S0)
european_call_price_1 = heston_call_put_mc(S_1, S0, r, T, 0,'C')
print("European Call Price under Heston (when correlation = -0.3): $",np.round(european_call_price_1,2))
european_put_price_1 = heston_call_put_mc(S_1, S0, r, T, 0,'P')
print("European Put Price under Heston (when correlation = -0.3): $",np.round(european_put_price_1,2))


European Call Price under Heston (when correlation = -0.3): $ 2.87
European Put Price under Heston (when correlation = -0.3): $ 2.82


In [ ]:
european_call_price_2 = heston_call_put_mc(S_2, S0, r, T, 0, 'C')
print("European Call Price under Heston (when correlation = -0.7): $",np.round(european_call_price_2,2))

european_put_price_2 = heston_call_put_mc(S_2, S0, r, T, 0,'P')
print("European Put Price under Heston (when correlation = -0.7): $",np.round(european_put_price_2,2))

European Call Price under Heston (when correlation = -0.7): $ 2.12
European Put Price under Heston (when correlation = -0.7): $ 3.45


Computing Delta and Gamma

In [ ]:
S0_up =100

In [ ]:
# Volatility process paths and Underlying price process paths when correlation is -0.3
#V_1_dash = SDE_vol(v0, kappa, theta, sigma, T, M, Ite, rand, 1, cho_matrix_1)
S_1_dash = Heston_paths(S0_up, r, V_1, 0, cho_matrix_1)

european_call_price = heston_call_put_mc(S_1_dash, S0, r, T, 0, 'C')
print("ATM European Call Price under Heston when underlying stock price is $100 (when correlation = -0.3): $",np.round(european_call_price,2))

european_put_price = heston_call_put_mc(S_1_dash, S0, r, T, 0, 'P')
print("ATM European Put Price under Heston when underlying stock price is $100 (when correlation = -0.3): $",np.round(european_put_price,3))

ATM European Call Price under Heston when underlying stock price is $100 (when correlation = -0.3): $ 19.86
ATM European Put Price under Heston when underlying stock price is $100 (when correlation = -0.3): $ 0.073


In [ ]:
dS =  S0 - S0_up
delta_call_rho3 = (european_call_price_1-european_call_price)/dS
delta_put_rho3 = (european_put_price_1-european_put_price)/dS
print("Delta for ATM European Call option: {:.4f}".format(delta_call_rho3))
print("Delta for ATM European Put option: {:.4f}".format(delta_put_rho3))

Delta for ATM European Call option: 0.8494
Delta for ATM European Put option: -0.1375


In [ ]:
# Volatility process paths and Underlying price process paths when correlation is -0.7
#V_2_dash = SDE_vol(v0, kappa, theta, sigma, T, M, Ite, rand, 1, cho_matrix_2)
S_2_dash = Heston_paths(S0_up, r, V_2, 0, cho_matrix_2)

european_call_price_rho7 = heston_call_put_mc(S_2_dash, S0, r, T, 0, 'C')
print("ATM European Call Price under Heston when underlying stock price is $100 (when correlation = -0.7): $",np.round(european_call_price_rho7,2))

european_put_price_rho7 = heston_call_put_mc(S_2_dash, S0, r, T, 0, 'P')
print("ATM European Put Price under Heston when underlying stock price is $100 (when correlation = -0.7): $",np.round(european_put_price_rho7,2))

ATM European Call Price under Heston when underlying stock price is $100 (when correlation = -0.7): $ 18.22
ATM European Put Price under Heston when underlying stock price is $100 (when correlation = -0.7): $ 0.16


In [ ]:
delta_call_rho7 = (european_call_price_2 - european_call_price_rho7)/dS
delta_put_rho7 = (european_put_price_2 - european_put_price_rho7)/dS
print("Delta for ATM European Call option: {:.4f}".format(delta_call_rho7))
print("Delta for ATM European Put option: {:.4f}".format(delta_put_rho7))

Delta for ATM European Call option: 0.8052
Delta for ATM European Put option: -0.1645


In [ ]:
S0_up_dash = 110
dS0 = S0 - S0_up
dS1 = S0_up - S0_up_dash

dS2 = dS0 - dS1

In [ ]:
#Gamma for correlation -0.3
S_3_dash = Heston_paths(S0_up_dash, r, V_1, 0, cho_matrix_1)

euro_call_price_1 = heston_call_put_mc(S_3_dash, S0, r, T, 0, 'C')
print("ATM European Call Price under Heston when underlying stock price is $110 (when correlation = -0.3): $",np.round(euro_call_price_1,2))
euro_put_price_1 = heston_call_put_mc(S_3_dash, S0, r, T, 0, 'P')
print("ATM European Put Price under Heston when underlying stock price is $110 (when correlation = -0.3): $",np.round(euro_put_price_1,3))

delta_call_rho3_2 = (european_call_price - euro_call_price_1) / (S0_up - S0_up_dash)
delta_put_rho3_2 = (european_put_price - euro_put_price_1) / (S0_up - S0_up_dash)

#print('Call ', delta_call_rho3_2)
#print('Put ',delta_put_rho3_2)

#print('call 1 ', delta_call_rho3)
#print('put 1 ',delta_put_rho3)

gamma_call_rho3 = (delta_call_rho3 - delta_call_rho3_2) / (dS2)
gamma_put_rho3 = (delta_put_rho3 - delta_put_rho3_2) / (dS2)

print("Gamma for ATM European call (when correlation = -0.3): {:.4f}".format(gamma_call_rho3))
print("Gamma for ATM European put (when correlation = -0.3): {:.4f}".format(gamma_put_rho3))

ATM European Call Price under Heston when underlying stock price is $110 (when correlation = -0.3): $ 29.66
ATM European Put Price under Heston when underlying stock price is $110 (when correlation = -0.3): $ 0.011
Gamma for ATM European call (when correlation = -0.3): 0.0131
Gamma for ATM European put (when correlation = -0.3): 0.0131


In [ ]:
#Gamma for correlation -0.7
S_4_dash = Heston_paths(S0_up_dash, r, V_2, 0, cho_matrix_2)

euro_call_price_2 = heston_call_put_mc(S_3_dash, S0, r, T, 0, 'C')
print("ATM European Call Price under Heston when underlying stock price is $110 (when correlation = -0.7): $",np.round(euro_call_price_1,2))
euro_put_price_2 = heston_call_put_mc(S_3_dash, S0, r, T, 0, 'P')
print("ATM European Put Price under Heston when underlying stock price is $110 (when correlation = -0.7): $",np.round(euro_put_price_1,3))

delta_call_rho7_2 = (european_call_price_rho7 - euro_call_price_2) / (S0_up - S0_up_dash)
delta_put_rho7_2 = (european_put_price_rho7 - euro_put_price_2) / (S0_up - S0_up_dash)

#print('Call ', delta_call_rho3_2)
#print('Put ',delta_put_rho3_2)

#print('call 1 ', delta_call_rho3)
#print('put 1 ',delta_put_rho3)

gamma_call_rho7 = (delta_call_rho7 - delta_call_rho7_2) / (dS2)
gamma_put_rho7 = (delta_put_rho7 - delta_put_rho7_2) / (dS2)

print("Gamma for ATM European call (when correlation = -0.7): {:.4f}".format(gamma_call_rho7))
print("Gamma for ATM European put (when correlation = -0.7): {:.4f}".format(gamma_put_rho7))

ATM European Call Price under Heston when underlying stock price is $110 (when correlation = -0.7): $ 29.66
ATM European Put Price under Heston when underlying stock price is $110 (when correlation = -0.7): $ 0.011
Gamma for ATM European call (when correlation = -0.7): 0.0339
Gamma for ATM European put (when correlation = -0.7): 0.0150


In [ ]:
data_options = {'Price (when corr = -0.3) ':['${:.2f}'.format(european_call_price_1), '${:.2f}'.format(european_put_price_1),],
                     'Price (when corr = -0.7)':['${:.2f}'.format(european_call_price_2), '${:.2f}'.format(european_put_price_2,2)],
                     'Delta (-0.3)':[round(delta_call_rho3,4), round(delta_put_rho3,4)],
                     'Gamma (-0.3)':[round(gamma_call_rho3,4), round(gamma_put_rho3,4)],
                     'Delta (-0.7)':[round(delta_call_rho7,4), round(delta_put_rho7,4)],
                     'Gamma (-0.7)':[round(gamma_call_rho7,4), round(gamma_put_rho7,4)]}
inx  = ["European Call", "European Put"]
pd.DataFrame(data_options, index = inx)

,Price (when corr = -0.3),Price (when corr = -0.7),Delta (-0.3),Gamma (-0.3),Delta (-0.7),Gamma (-0.7)
European Call,$2.87,$2.12,0.8494,0.0131,0.8052,0.0339
European Put,$2.82,$3.45,-0.1375,0.0131,-0.1645,0.0150


Validating Put-Call Parity

In [ ]:
#When Correlation is -0.3
# Put-call parity under HS method:
#print(european_call_price_1, european_put_price_1)
left_side = european_call_price_1 + S0 * np.exp(-r * T)
print(S0 * np.exp(-r * T))
print(f"left_side: {round(left_side,2)}")
right_side = european_put_price_1 + S0
print(f"right_side: {round(right_side,2)}")

if round(left_side,2) == round(right_side,2):
  print("Put-call parity holds under Heston method.")
else:
  print("Put-call parity does not hold under Heston method.")

78.90752795736353
left_side: 81.78
right_side: 82.83
Put-call parity does not hold under Heston method.


### Jump Modeler

In [ ]:
S0 = 80
K = S0

r = 0.055
sigma = 0.35
T = 3/12 # 3 months
M = 90
dt = T / M

lambda_8 = 0.75
lambda_9 = 0.25

mu = -0.5
delta = 0.22
iter = 10000

In [ ]:
def european_merton_mc(S0, K, r, sigma, T, M, Iter, lamb, mu, delta, opttype):

    dt = T / M
    SM = np.zeros((M+1, Iter))
    SM[0] = S0

    rj = lamb * (np.exp(mu + 0.5 * delta**2) - 1)

    # Random numbers
    z1 = np.random.standard_normal((M + 1, Iter))
    z2 = np.random.standard_normal((M + 1, Iter))
    y = np.random.poisson(lamb * dt, (M + 1, Iter))

    # Simulating prices
    for t in range(1, M + 1):
        SM[t] = SM[t - 1] * (
            np.exp((r - rj - 0.5 * sigma**2) * dt + sigma * np.sqrt(dt) * z1[t])
            + (np.exp(mu + delta * z2[t]) - 1) * y[t]
        )
        SM[t] = np.maximum(SM[t], 0.00001)

    # Option prices
    if opttype == 'call':
        arr = SM[-1] - K
    elif opttype == 'put':
        arr = K - SM[-1]
    else:
        raise ValueError(f'Unknown option type {opttype}')

    ST = np.where(arr < 0, 0, arr).mean() * np.exp(-r * T)
    return ST

Pricing an ATM european call and put options with jump intensity parameter equal to 0.75

In [ ]:
euro_call_merton_mc8 = european_merton_mc(S0, K, r, sigma, T, M, iter, lambda_8, mu, delta, "call")
euro_put_merton_mc8 = european_merton_mc(S0, K, r, sigma, T, M, iter, lambda_8, mu, delta, "put")
print("European call ATM option price using Merton model: {}".format(round(euro_call_merton_mc8,2)))
print("European put ATM option price using Merton model: {}".format(round(euro_put_merton_mc8,2)))

European call ATM option price using Merton model: 8.22
European put ATM option price using Merton model: 7.09


Pricing an ATM european call and put options with jump intensity parameter equal to 0.25

In [ ]:
euro_call_merton_mc9 = european_merton_mc(S0, K, r, sigma, T, M, iter, lambda_9, mu, delta, "call")
euro_put_merton_mc9 = european_merton_mc(S0, K, r, sigma, T, M, iter, lambda_9, mu, delta, "put")
print("European call ATM option price using Merton model: {}".format(round(euro_call_merton_mc9,2)))
print("European put ATM option price using Merton model: {}".format(round(euro_put_merton_mc9,2)))

European call ATM option price using Merton model: 7.02
European put ATM option price using Merton model: 5.87


Calculation delta and gamma for options

In [ ]:
# Stock price changes
stock_price_delta = 10
S1 = S0 + stock_price_delta
dS = S1 - S0

# Calcalation of deltas for put and call with lambda 0.75
euro_call_merton_mc8_0 = european_merton_mc(S0, S0, r, sigma, T, M, iter, lambda_8, mu, delta, "call")
euro_call_merton_mc8_1 = european_merton_mc(S1, K, r, sigma, T, M, iter, lambda_8, mu, delta, "call")
delta_call_8 = (euro_call_merton_mc8_1 - euro_call_merton_mc8_0) / dS

euro_put_merton_mc8_0 = european_merton_mc(S0, S0, r, sigma, T, M, iter, lambda_8, mu, delta, "put")
euro_put_merton_mc8_1 = european_merton_mc(S1, K, r, sigma, T, M, iter, lambda_8, mu, delta, "put")
delta_put_8 = (euro_put_merton_mc8_1 - euro_put_merton_mc8_0) / dS

print(f"Delta for ATM European Call option with lambda 0.75 (Merton model): {round(delta_call_8,4)}")
print(f"Delta for ATM European Put option with lambda 0.75 (Merton model): {round(delta_put_8,4)}")

# Calcalation of deltas for put and call with lambda 0.25
euro_call_merton_mc9_0 = european_merton_mc(S0, S0, r, sigma, T, M, iter, lambda_9, mu, delta, "call")
euro_call_merton_mc9_1 = european_merton_mc(S1, K, r, sigma, T, M, iter, lambda_9, mu, delta, "call")
delta_call_9 = (euro_call_merton_mc8_1 - euro_call_merton_mc8_0) / dS

euro_put_merton_mc9_0 = european_merton_mc(S0, S0, r, sigma, T, M, iter, lambda_9, mu, delta, "put")
euro_put_merton_mc9_1 = european_merton_mc(S1, K, r, sigma, T, M, iter, lambda_9, mu, delta, "put")
delta_put_9 = (euro_put_merton_mc8_1 - euro_put_merton_mc8_0) / dS

print("-------------------------------------------------------")
print(f"Delta for ATM European Call option with lambda 0.25 (Merton model): {round(delta_call_9,4)}")
print(f"Delta for ATM European Put option with lambda 0.25 (Merton model): {round(delta_put_9,4)}")

Delta for ATM European Call option with lambda 0.75 (Merton model): 0.7454
Delta for ATM European Put option with lambda 0.75 (Merton model): -0.2485
-------------------------------------------------------
Delta for ATM European Call option with lambda 0.25 (Merton model): 0.7454
Delta for ATM European Put option with lambda 0.25 (Merton model): -0.2485


In [ ]:
# Stock price changes
stock_price_delta0 = 10
stock_price_delta1 = 11
S1 = S0 + stock_price_delta0
S2 = S0 + stock_price_delta1
dS0 = S1 - S0
dS1 = S2 - S1
d2S = dS1 - dS0


# Calcalation of gammas for put and call with lambda 0.75
euro_call_merton_mc8_0 = european_merton_mc(S0, K, r, sigma, T, M, iter, lambda_8, mu, delta, "call")
euro_call_merton_mc8_1 = european_merton_mc(S1, K, r, sigma, T, M, iter, lambda_8, mu, delta, "call")
euro_call_merton_mc8_2 = european_merton_mc(S2, K, r, sigma, T, M, iter, lambda_8, mu, delta, "call")
delta_call_8_0 = (euro_call_merton_mc8_1 - euro_call_merton_mc8_0) / dS0
delta_call_8_1 = (euro_call_merton_mc8_2 - euro_call_merton_mc8_1) / dS1
gamma_call8 = (delta_call_8_1 - delta_call_8_0) / d2S

euro_put_merton_mc8_0 = european_merton_mc(S0, K, r, sigma, T, M, iter, lambda_8, mu, delta, "put")
euro_put_merton_mc8_1 = european_merton_mc(S1, K, r, sigma, T, M, iter, lambda_8, mu, delta, "put")
euro_put_merton_mc8_2 = european_merton_mc(S2, K, r, sigma, T, M, iter, lambda_8, mu, delta, "put")
delta_put_8_0 = (euro_put_merton_mc8_1 - euro_put_merton_mc8_0) / dS0
delta_put_8_1 = (euro_put_merton_mc8_2 - euro_put_merton_mc8_1) / dS1
gamma_put8 = (delta_put_8_1 - delta_put_8_0) / d2S

print(f"Gamma for ATM European Call option with lambda 0.75 (Merton model): {round(gamma_call8,4)}")
print(f"Gamma for ATM European Put option with lambda 0.75 (Merton model): {round(gamma_put8,4)}")

# Calcalation of gammas for put and call with lambda 0.25
euro_call_merton_mc9_0 = european_merton_mc(S0, K, r, sigma, T, M, iter, lambda_9, mu, delta, "call")
euro_call_merton_mc9_1 = european_merton_mc(S1, K, r, sigma, T, M, iter, lambda_9, mu, delta, "call")
euro_call_merton_mc9_2 = european_merton_mc(S2, K, r, sigma, T, M, iter, lambda_9, mu, delta, "call")
delta_call_9_0 = (euro_call_merton_mc9_1 - euro_call_merton_mc9_0) / dS0
delta_call_9_1 = (euro_call_merton_mc9_2 - euro_call_merton_mc9_1) / dS1
gamma_call9 = (delta_call_9_1 - delta_call_9_0) / d2S

euro_put_merton_mc9_0 = european_merton_mc(S0, K, r, sigma, T, M, iter, lambda_9, mu, delta, "put")
euro_put_merton_mc9_1 = european_merton_mc(S1, K, r, sigma, T, M, iter, lambda_9, mu, delta, "put")
euro_put_merton_mc9_2 = european_merton_mc(S2, K, r, sigma, T, M, iter, lambda_9, mu, delta, "put")
delta_put_9_0 = (euro_put_merton_mc9_1 - euro_put_merton_mc9_0) / dS0
delta_put_9_1 = (euro_put_merton_mc9_2 - euro_put_merton_mc9_1) / dS1
gamma_put9 = (delta_put_9_1 - delta_put_9_0) / d2S

print(f"Gamma for ATM European Call option with lambda 0.25 (Merton model): {round(gamma_call9,4)}")
print(f"Gamma for ATM European Put option with lambda 0.25 (Merton model): {round(gamma_put9,4)}")

Gamma for ATM European Call option with lambda 0.75 (Merton model): -0.0132
Gamma for ATM European Put option with lambda 0.75 (Merton model): -0.0093
Gamma for ATM European Call option with lambda 0.25 (Merton model): -0.0155
Gamma for ATM European Put option with lambda 0.25 (Merton model): -0.0333


Summarizing in a table (for different lambdas: 0.75 and 0.25):

In [ ]:
data = {"Price (0.75)": ['${:.2f}'.format(euro_call_merton_mc8), '${:.2f}'.format(euro_put_merton_mc8)],
        "Price (0.25)": ['${:.2f}'.format(euro_call_merton_mc9), '${:.2f}'.format(euro_put_merton_mc9)],
        "Delta (0.75)": [round(delta_call_8,4), round(delta_put_8,4)],
        "Delta (0.25)": [round(delta_call_9,4), round(delta_put_9,4)],
        "Gamma (0.25)": [round(gamma_call8,4), round(gamma_put8,4)],
        "Gamma (0.75)": [round(gamma_call9,4), round(gamma_put9,4)]}
idx_names = ["Call option", "Put option"]

pd.DataFrame(data, index = idx_names)

,Price (0.75),Price (0.25),Delta (0.75),Delta (0.25),Gamma (0.25),Gamma (0.75)
Call option,$8.22,$7.02,0.7454,0.7454,-0.0132,-0.0155
Put option,$7.09,$5.87,-0.2485,-0.2485,-0.0093,-0.0333


### Model Validator

#### 11. For Questions 5, 6, 8, and 9, use put-call Parity to determine if the prices of the put and call from the Heston Model and Merton Model satisfy put-call parity.

In [ ]:
def verify_put_call_parity(S0, C, P, K, r, T, tolerance=2):
    left = C + K*np.exp(-r*T)
    right = P + S0

    print(f'left hand side: {left:.2f}')
    print(f'right hand side: {right:.2f}')

    if abs(left-right) < tolerance:
        print('Put-call parity holds')
    else:
        print('Put-call parity does not hold')

##### Using the Heston Model and Monte-Carlo simulation, price an ATM European call and put, using a correlation value of -0.30.

In [ ]:
verify_put_call_parity(S0=S0, C=european_call_price_1, P=european_put_price_1, K=S0, r=r, T=T)

left hand side: 81.78
right hand side: 82.83
Put-call parity holds


##### Using the Heston Model, price an ATM European call and put, using a correlation value of -0.70.

In [ ]:
verify_put_call_parity(S0=S0, C=european_call_price_2, P=european_call_price_2, K=S0, r=r, T=T)

left hand side: 81.03
right hand side: 82.12
Put-call parity holds


##### Using the Merton Model, price an ATM European call and put with jump intensity parameter equal to 0.75.


In [ ]:
verify_put_call_parity(S0=S0, C=euro_call_merton_mc8_0, P=euro_put_merton_mc8_0, K=S0, r=r, T=T)

left hand side: 87.26
right hand side: 87.26
Put-call parity holds


##### Using the Merton Model, price an ATM European call and put with jump intensity parameter equal to 0.25

In [ ]:
verify_put_call_parity(S0=S0, C=euro_call_merton_mc9_0, P=euro_put_merton_mc9_0, K=S0, r=r, T=T)

left hand side: 85.63
right hand side: 85.67
Put-call parity holds


#### 12. Run the Heston Model and Merton Model for 7 different strikes: 3 OTM calls; 1 ATM call; and 3 ITM calls. The strikes should be equally spaced. Try to use the following APPROXIMATE moneyness values: 0.85, 0.90, 0.95, 1, 1.05, 1.10, and 1.15. Recall that moneyness = stock/strike.

In [ ]:
moneyness_values = [0.85, 0.9, 0.95, 1, 1.05, 1.10, 1.15]

data = []

for moneyness in moneyness_values:

    K = S0 / moneyness

    # use a correlation value of -0.3
    hetson_call = heston_call_put_mc(S_1, K, r, T, 0, 'C')
    hetson_put = heston_call_put_mc(S_1, K, r, T, 0,'P')

    # jump intensity parameter equal to 0.75
    merton_call = european_merton_mc(S0, K, r, sigma, T, M, iter, lambda_8, mu, delta, 'call')
    merton_put = european_merton_mc(S0, K, r, sigma, T, M, iter, lambda_8, mu, delta, 'put')

    data.append([hetson_call, hetson_put, merton_call, merton_put])

df = pd.DataFrame(np.array(data).T, index=['hetson_call', 'hetson_put', 'merton_call', 'merton_put'], columns=moneyness_values)
pd.set_option('display.precision', 2)
df


,0.85,0.90,0.95,1.00,1.05,1.10,1.15
hetson_call,0.10,0.41,1.25,2.87,5.18,7.86,10.63
hetson_put,13.98,9.13,5.36,2.83,1.38,0.64,0.29
merton_call,2.71,4.43,6.12,8.24,10.41,12.99,14.82
merton_put,15.60,11.97,9.54,7.13,5.73,4.58,3.85


## **Step 2**

Question 13

In [ ]:
#Redeclaring variables
S0_amer = 80
T = 1/4 # 3 months
M_amer= int(T * 255)
v0_amer= 0.032
Ite_amer = 1000000

In [ ]:
def heston_american_call_mc(S, K, r, T, t):
    payoff = np.zeros((M_amer + 1, Ite_amer), dtype=float)

    #payoff = np.maximum(0, S[-1, :] - K)
    # Option princing
    for i in range(Ite_amer):
        payoff[-1, i] = max(S[-1, i] - K, 0) # Option price at maturity

    for j in range(M_amer-1, -1, -1):
        for it in range(Ite_amer):
            payoff[j, it] = max(S[j, it] - K,
                             np.exp(-r*dt) * payoff[j+1, it]) # maximum of payoff and option price (discounted)


    average = np.mean(payoff)

    return average

In [ ]:
V_Amer = SDE_vol(v0_amer, kappa, theta, sigma, T, M, Ite_amer, rand, 1, cho_matrix_1)
S_Amer = Heston_paths(S0_amer, r, V_Amer, 0, cho_matrix_1)
print(M)
american_call_heston = heston_american_call_mc(S_Amer, S0, r, T, 0)
print("ATM American call price using heston (when corr = -0.3): ${:.2f}".format(american_call_heston))

63
ATM American call price using heston (when corr = -0.3): $4.40


In [ ]:
def merton_american_call_mc(S, K, r, sigma, T, M, iter, lamb, mu, delta):
    dt = T / M

    rj = lamb * (np.exp(mu + 0.5 * delta**2) - 1)

    # Random numbers
    z1 = np.random.standard_normal((M + 1, iter))
    z2 = np.random.standard_normal((M + 1, iter))
    y = np.random.poisson(lamb * dt, (M + 1, iter))

    # Monte-Carlo Simulation of underlying prices
    underlying = np.zeros((M+1, iter))
    underlying[0] = S
    for t in range(1, M + 1):
        underlying[t] = underlying[t - 1] * (
            np.exp((r - rj - 0.5 * sigma**2) * dt + sigma * np.sqrt(dt) * z1[t])
            + (np.exp(mu + delta * z2[t]) - 1) * y[t]
        )
        underlying[t] = np.maximum(underlying[t], 0.00001)

    # American call option prices
    prc = np.zeros((M+1, iter))

    # Option princing
    for i in range(iter):
        prc[-1, i] = max(underlying[-1, i] - K, 0) # Option price at maturity

    for j in range(M-1, -1, -1):
        for it in range(iter):
            prc[j, it] = max(underlying[j, it] - K,
                             np.exp(-r*dt) * prc[j+1, it]) # maximum of payoff and option price (discounted)

    return sum(prc[0,:]) / iter

In [ ]:
merton_call_price_mc = merton_american_call_mc(S0, K, r, sigma, T, M, iter, lambda_8, mu, delta)
print(f"American call option price (Merton model) with Monte Carlo simulation: {round(merton_call_price_mc,2)}")

American call option price (Merton model) with Monte Carlo simulation: 14.21


Comments:

Question 14

In [ ]:
S0 = 80
r = 0.055
sigma = 0.35
T = 1/4 # 3 months
K_uai = 95
B_uai = 95

v0 = 0.032
kappa = 1.85
theta = 0.045
M=int(T * 255)
Ite = 1000000
dt = T/M

In [ ]:
def uai_heston_call_mc(S, K, B, r, T, t):
  is_up_and_in_payoff = np.zeros((M+1,Ite))

  for i in range(M+1):
    is_up_and_in_payoff[i] = np.where(S[i] >= B, S[i]-K, 0)


  average = np.exp(-r *T)*is_up_and_in_payoff
  average = np.mean(average)
  print(average)
  return average


In [ ]:
V_ = SDE_vol(v0, kappa, theta, sigma, T, M, Ite, rand, 1, cho_matrix_2)

S_uai = Heston_paths(S0, r, V_, 0, cho_matrix_2)

In [ ]:
#print(S_uai)
#print(S_uai[-1].shape)
vanilla_price = heston_call_put_mc(S_uai, K_uai, r, T, 0, 'C')
price_uai = uai_heston_call_mc(S_uai, K_uai, B_uai, r, T, 0)

print("Vanilla European Call (when corr = -0.7):${:.2f}".format(vanilla_price)) #Already calculated for ques 5
print("UAI Option price (when corr = -0.7): ${:.4f}".format(price_uai))

0.0011987353033436879
Vanilla European Call (when corr = -0.7):$0.01
UAI Option price (when corr = -0.7): $0.0012


##### Question 15.

In [ ]:
S0 = 80
DAI_barrier = 65
DAI_K = 65

In [ ]:
def dai_put_merton_mc(S0, K, r, sigma, T, M, Iter, lamb, mu, delta, barrier):

    dt = T / M
    SM = np.zeros((M+1, Iter))
    IS_IN = np.zeros((M+1, Iter))
    IS_IN[0] = np.where(S0 <= barrier, 1, 0)
    SM[0] = S0

    rj = lamb * (np.exp(mu + 0.5 * delta**2) - 1)

    # Random numbers
    z1 = np.random.standard_normal((M + 1, Iter))
    z2 = np.random.standard_normal((M + 1, Iter))
    y = np.random.poisson(lamb * dt, (M + 1, Iter))

    # Simulating prices
    for t in range(1, M + 1):
        SM[t] = SM[t - 1] * (
            np.exp((r - rj - 0.5 * sigma**2) * dt + sigma * np.sqrt(dt) * z1[t])
            + (np.exp(mu + delta * z2[t]) - 1) * y[t]
        )
        SM[t] = np.maximum(SM[t], 0.00001)
        IS_IN[t] = np.where(SM[t] <= barrier, 1, IS_IN[t-1])

    arr = K - SM[-1]

    ST = (np.where(arr < 0, 0, arr)*IS_IN).mean() * np.exp(-r * T)
    return ST

In [ ]:
q15_simple_european_put = european_merton_mc(S0, DAI_K, r, sigma, T, M, iter, lambda_8, mu, delta, "put")
q15_dai_put = dai_put_merton_mc(S0, DAI_K, r, sigma, T, M, iter, lambda_8, mu, delta, DAI_barrier)

In [ ]:
print(f'simple european put: {q15_simple_european_put:.2f}')
print(f'DAI put: {q15_dai_put:.2f}')

simple european put: 2.81
DAI put: 1.49
